In [1]:
# Main idea for Carvana competition: 
## A. the pictures are grouped by car to avoid leakage in the validation set
## B. background is separated using USV and reused as 4th layer in  
## C. the RESIZE of pictures at input or output induces some loss 
## so the pictures are cut on top and bottom and extended with a black screen on left and right

In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import cv2
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from sklearn.model_selection import train_test_split

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import numpy as np
% matplotlib inline
import glob

Using TensorFlow backend.
/home/xavierc/.local/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [10]:
from model.u_net import get_unet_128, get_unet_256, get_unet_512, get_unet_1024, get_unet_tf_1536x1024_crop \
                        ,get_tiram, get_unet_1024_usv, get_unet_tf_1152x1152_usv_crop, get_unet_tf_1280x1280_usv_crop \
                         , get_unet_tf4_1536x1280_usv_bl_crop

#input_size = (1536,1024)
input_size = (1408,1280)

max_epochs = 100
batch_size = 3

orig_width = 1918
orig_height = 1280

threshold = 0.5

#model = get_unet_tf_1536x1024_crop()


model = get_unet_tf4_1536x1280_usv_bl_crop((1280, 1536, 4))

In [14]:
input_size_w = int(input_size[0])
input_size_h = int(input_size[1])
epochs = max_epochs

print(input_size_w,input_size_h)

(1408, 1280)


In [ ]:
## read csv with pictures

## the 

df_train = pd.read_csv('/opt/datasets/kaggle/Carvana/train_masks.csv')
ids_train = df_train['img'].map(lambda s: s.split('.')[0])
df_train = df_train.sort_values(by='img').reset_index(drop=True)
# df_train = df_train.sample(frac=1).reset_index(drop=True)
df_train['img_path'] = df_train['img'].apply(lambda x: '/opt/datasets/kaggle/Carvana/train/'+x)
df_train['mask_path'] = df_train['img'].apply(lambda x: '/opt/datasets/kaggle/Carvana/train_masks_png/'+x.split('.')[0]+'_mask.png')
df_train['usv_path'] = df_train['img'].apply(lambda x: '/opt/datasets/kaggle/Carvana/backgdusv2/'+x.split('.')[0]+'.png')
df_train= df_train.drop('rle_mask', 1)
df_train.head()

In [17]:
# simple cross validation to start.

trn_data = df_train[:4370]
val_data = df_train[4370:]

print('Training on {} samples'.format(len(trn_data)))
print('Validating on {} samples'.format(len(val_data)))

Training on 4370 samples
Validating on 718 samples


In [24]:
# various function to augment the pictures and the mask

def randomHueSaturationValue(image, hue_shift_limit=(-180, 180),
                             sat_shift_limit=(-255, 255),
                             val_shift_limit=(-255, 255), u=0.5):
    if np.random.random() < u:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(image)
        hue_shift = np.random.uniform(hue_shift_limit[0], hue_shift_limit[1])
        h = cv2.add(h, hue_shift)
        sat_shift = np.random.uniform(sat_shift_limit[0], sat_shift_limit[1])
        s = cv2.add(s, sat_shift)
        val_shift = np.random.uniform(val_shift_limit[0], val_shift_limit[1])
        v = cv2.add(v, val_shift)
        image = cv2.merge((h, s, v))
        image = cv2.cvtColor(image, cv2.COLOR_HSV2BGR)

    return image


def randomShiftScaleRotate(image, mask,
                           shift_limit=(-0.0625, 0.0625),
                           scale_limit=(-0.1, 0.1),
                           rotate_limit=(-45, 45), aspect_limit=(0, 0),
                           borderMode=cv2.BORDER_CONSTANT, u=0.5):
    if np.random.random() < u:
        height, width, channel = image.shape
        
        add_height = int(height / 3)
        add_width = int(width / 3)
        image = cv2.copyMakeBorder(image,add_height,add_height,add_width,add_width,cv2.BORDER_REFLECT_101) #top, bottom, left, right 
        mask = cv2.copyMakeBorder(mask,add_height,add_height,add_width,add_width,cv2.BORDER_REFLECT_101) #top, bottom, left, right 
        height, width, channel = image.shape
        
        angle = np.random.uniform(rotate_limit[0], rotate_limit[1])*1.0  # degree
        scale = np.random.uniform(1 + scale_limit[0], 1 + scale_limit[1])
        aspect = np.random.uniform(1 + aspect_limit[0], 1 + aspect_limit[1])
        sx = scale * aspect / (aspect ** 0.5)
        sy = scale / (aspect ** 0.5)
        dx = round(np.random.uniform(shift_limit[0], shift_limit[1]) * width)
        dy = round(np.random.uniform(shift_limit[0], shift_limit[1]) * height)

        cc = np.math.cos(angle / 180 * np.math.pi) * sx
        ss = np.math.sin(angle / 180 * np.math.pi) * sy
        rotate_matrix = np.array([[cc, -ss], [ss, cc]])

        box0 = np.array([[0, 0], [width, 0], [width, height], [0, height], ])
        box1 = box0 - np.array([width / 2, height / 2])
        box1 = np.dot(box1, rotate_matrix.T) + np.array([width / 2 + dx, height / 2 + dy])

        box0 = box0.astype(np.float32)
        box1 = box1.astype(np.float32)
        mat = cv2.getPerspectiveTransform(box0, box1)
        image = cv2.warpPerspective(image, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                    borderValue=(
                                        0, 0,
                                        0,))
        image = image[add_height:-add_height,add_width:-add_width,:]
        mask = cv2.warpPerspective(mask, mat, (width, height), flags=cv2.INTER_LINEAR, borderMode=borderMode,
                                   borderValue=(
                                       0, 0,
                                       0,))
        mask = mask[add_height:-add_height,add_width:-add_width]

    return image, mask


def randomHorizontalFlip(image, mask, u=0.5):
    if np.random.random() < u:
        image = cv2.flip(image, 1)
        mask = cv2.flip(mask, 1)
#     if np.random.random() < u:
#         image = cv2.flip(image, 2)
#         mask = cv2.flip(mask, 2)
#     if np.random.random() < u:
#         image = np.rot90(image,1)
#         mask = np.rot90(mask,1)
#     if np.random.random() < u:
#         image = np.rot90(image,2)
#         mask = np.rot90(mask,2)
    return image, mask



def mask_jitter(mask):
    w,h = mask.shape
    v1 = np.random.randint(-1,2)

    if v1 ==-1:
        mask = cv2.copyMakeBorder(mask,1,0,0,0,cv2.BORDER_REFLECT_101)
        mask = mask[:w,:]

    if v1 ==1:
        mask = cv2.copyMakeBorder(mask,0,1,0,0,cv2.BORDER_REFLECT_101)
        mask = mask[1:,:]

        
    v2 = np.random.randint(-1,2)
    if v2 ==-1:
        mask = cv2.copyMakeBorder(mask,0,0,1,0,cv2.BORDER_REFLECT_101)
        mask = mask[:,:h]

    if v2==1:
        mask = cv2.copyMakeBorder(mask,0,0,0,1,cv2.BORDER_REFLECT_101)
        mask = mask[:,1:]

    return mask


from dehaze import DarkChannel, AtmLight, TransmissionEstimate, TransmissionRefine, Recover

def dehaze_img(img):
    I = img.astype('float64')/255.;

    dark = DarkChannel(I,15);
    A = AtmLight(I,dark);
    te = TransmissionEstimate(I,A,15);
    t = TransmissionRefine(img,te);
    J = Recover(I,t,A,0.1);
    return np.array(J*255.)

In [25]:
## customized generator for both train and test


def train_generator(data, batch_size):

    while 1:
        x_batch = []
        y_batch = []
        
        for i_batch in range(batch_size):
            i_line = np.random.randint(len(data))
            
            img = cv2.imread(data.img_path.iloc[i_line])
#             img = cv2.fastNlMeansDenoisingColored(img,None,5,5,5,5)
            img = cv2.resize(img, (input_size_w, input_size_h),cv2.INTER_AREA)
            img = cv2.copyMakeBorder(img,0,0,64,64,cv2.BORDER_REFLECT_101) #top, bottom, left, right
            usv = cv2.imread(data.usv_path.iloc[i_line], cv2.IMREAD_GRAYSCALE)
            usv = cv2.resize(usv, (input_size_w, input_size_h),cv2.INTER_AREA)
            usv = cv2.copyMakeBorder(usv,0,0,64,64,cv2.BORDER_REFLECT_101)
            
            mask = cv2.imread(data.mask_path.iloc[i_line], cv2.IMREAD_GRAYSCALE)
#             if mask.max() <=1:
#                 mask[mask<.5]=0
#                 mask[mask>.5]=255
# #                 mask = mask*255
            mask = cv2.resize(mask, (input_size_w, input_size_h) ,cv2.INTER_AREA)
            mask = cv2.copyMakeBorder(mask,0,0,64,64,cv2.BORDER_REFLECT_101)
            
            img = randomHueSaturationValue(img,
                                           hue_shift_limit=(-1, 1),
                                           sat_shift_limit=(-1, 1),
                                           val_shift_limit=(-1, 1))
            
            img, mask = randomShiftScaleRotate(img, mask,
                                               shift_limit=(-0.03, 0.03),
                                               scale_limit=(-0.00, 0.00),
                                               rotate_limit=(-0,0))
            
#             imgdeh = DarkChannel(img,9)
    
            #### concatenate img and usv in a 4D tensor
            img = np.dstack((img, usv)) 
            
            
            img, mask = randomHorizontalFlip(img, mask)
            
            mask = mask[:,64:-64]
#             mask = mask_jitter(mask)
            mask = np.expand_dims(mask, axis=2)

            
            x_batch.append(img)
            y_batch.append(mask)
            #print(mask.shape, mask) 
        x_batch = np.array(x_batch, np.float32) /255. - .5
        y_batch = np.array(y_batch, np.float32) / 255
#         x_batch = x_batch.transpose((0,3,1,2))
#         y_batch = y_batch.transpose((0,3,1,2))
        yield x_batch, y_batch


def valid_generator(data):
    
    while 1:
        x_batch = []
        y_batch = []
        
        for i_batch in range(batch_size):
            i_line = np.random.randint(len(data))
            
            img = cv2.imread(data.img_path.iloc[i_line])
#             img = cv2.fastNlMeansDenoisingColored(img,None,5,5,5,5)
            img = cv2.resize(img, (input_size_w, input_size_h),cv2.INTER_AREA)
            img = cv2.copyMakeBorder(img,0,0,64,64,cv2.BORDER_REFLECT_101)
        
            usv = cv2.imread(data.usv_path.iloc[i_line], cv2.IMREAD_GRAYSCALE)
            usv = cv2.resize(usv, (input_size_w, input_size_h))
            usv = cv2.copyMakeBorder(usv,0,0,64,64,cv2.BORDER_REFLECT_101)
            
#             imgdeh = DarkChannel(img,9)
            
            img = np.dstack((img, usv))
            
            mask = cv2.imread(data.mask_path.iloc[i_line], cv2.IMREAD_GRAYSCALE)
#             if mask.max() <=1:
#                 mask[mask<.5]=0
#                 mask[mask>.5]=255
            mask = cv2.resize(mask, (input_size_w, input_size_h),cv2.INTER_AREA)
            mask = cv2.copyMakeBorder(mask,0,0,64,64,cv2.BORDER_REFLECT_101)
            
            mask = mask[:,64:-64]
            
            mask = np.expand_dims(mask, axis=2)
#             img = dehaze_img(img)

            x_batch.append(img)
            y_batch.append(mask)
        
        x_batch = np.array(x_batch, np.float32) /255. - .5
        y_batch = np.array(y_batch, np.float32) / 255
#         x_batch = x_batch.transpose((0,3,1,2))
#         y_batch = y_batch.transpose((0,3,1,2))
        yield x_batch, y_batch

In [26]:
## customized generator to potentially use test results for pseudo labeling method.
## can also be used to rebalance the train set

def trn_gen_mix(data, prop=0.67):

    n1 = int(batch_size*prop)
    n2 = batch_size - n1
    sg1 = train_generator(trn_data, n1)
    sg2 = train_generator(df_test, n2)
    while True:
        out1 = sg1.next()
        out2 = sg2.next()
        if out2 is None:
            yield out1
        else:
            yield np.concatenate((out1[0], out2[0])), np.concatenate((out1[1], out2[1]))

In [33]:
callbacks = [EarlyStopping(monitor='val_dice_metric',
                           patience=6,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_metric',
                               factor=0.1,
                               patience=3,
                               verbose=1,
                               epsilon=5e-2,
                               mode='max'),
             ModelCheckpoint(monitor='val_dice_metric',
                             filepath='/home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/best_weights_1280x1280_unet_th_usv_crop_t0053b.hdf5',
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max'),
            TensorBoard(log_dir='logs')]

#model.summary()

model.load_weights('/home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/best_weights_1280x1280_unet_th_usv_crop_t0053b.hdf5')

In [42]:
batch_size = 3

In [43]:
from keras.optimizers import RMSprop, Adam, Adamaccum, SGDAccum
model.optimizer=Adamaccum(lr=1e-5, decay=1e-6)

model.fit_generator(generator=train_generator(trn_data,batch_size),
                    steps_per_epoch=np.ceil(float(len(trn_data)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(val_data),
                    validation_steps=np.ceil(float(len(val_data)) / float(batch_size)))

Epoch 1/100
1457/1457 [==============================] - 1766s - loss: -0.9895 - dice_metric: 0.9947 - val_loss: -0.9900 - val_dice_metric: 0.9950
Epoch 2/100
1457/1457 [==============================] - 1753s - loss: -0.9896 - dice_metric: 0.9948 - val_loss: -0.9900 - val_dice_metric: 0.9950
Epoch 3/100
1457/1457 [==============================] - 1702s - loss: -0.9899 - dice_metric: 0.9949 - val_loss: -0.9902 - val_dice_metric: 0.9951
Epoch 4/100
1457/1457 [==============================] - 1700s - loss: -0.9900 - dice_metric: 0.9950 - val_loss: -0.9901 - val_dice_metric: 0.9950
Epoch 5/100
1456/1457 [============================>.] - ETA: 1s - loss: -0.9899 - dice_metric: 0.9949
Epoch 00004: reducing learning rate to 9.99999974738e-07.
1457/1457 [==============================] - 1698s - loss: -0.9899 - dice_metric: 0.9949 - val_loss: -0.9900 - val_dice_metric: 0.9950
Epoch 6/100
1457/1457 [==============================] - 1697s - loss: -0.9899 - dice_metric: 0.9949 - val_loss: -0.

In [15]:
# Epoch 00012: reducing learning rate to 9.99999974738e-07.
# 1018/1018 [==============================] - 1292s - loss: -0.9937 - dice_metric: 0.9968 - val_loss: -0.9936 - val_dice_metric: 0.9968
# Epoch 00012: early stopping

In [14]:
# simple fold -- test generator

In [8]:
weights_path = '/home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/'

model1 = get_unet_tf_1280x1280_usv_crop()
model1.load_weights(weights_path+'unettfcv1280x1289_crop_fold_1.hdf5')
model2 = get_unet_tf_1280x1280_usv_crop()
model2.load_weights(weights_path+'unettfcv1280x1289_crop_fold_2.hdf5')
model3 = get_unet_tf_1280x1280_usv_crop()
model3.load_weights(weights_path+'unettfcv1280x1289_crop_fold_3.hdf5')
model4 = get_unet_tf_1280x1280_usv_crop()
model4.load_weights(weights_path+'unettfcv1280x1289_crop_fold_4.hdf5')
model5 = get_unet_tf_1280x1280_usv_crop()
model5.load_weights(weights_path+'unettfcv1280x1289_crop_fold_5.hdf5')

In [ ]:
# carvana_path = '/opt/datasets/kaggle/Carvana/'
# testmasks_fld =  sorted(glob.glob(carvana_path + 'test/*'))
# len(testmasks_fld), testmasks_fld[:2]

# df_test = pd.DataFrame()
# df_test['img_path'] = testmasks_fld
# df_test['img'] = df_test['img_path'].apply(lambda x: x.split('/')[-1])
# df_test['mask_path'] = df_test['img'].apply(lambda x: '/opt/datasets/kaggle/Carvana/test_masks/'+x+'.png')
# df_test = df_test.sort_values(by='img').reset_index(drop=True)
# # df_train = df_train.sample(frac=1).reset_index(drop=True)

# df_test['usv_path'] = df_test['img'].apply(lambda x: '/opt/datasets/kaggle/Carvana/backgdusv2/'+x.split('.')[0]+'.png')

# df_test.tail()

In [ ]:
import threading
import multiprocessing as queue
import tensorflow as tf
from tqdm import tqdm

batch_size = 3

weight_path = '/home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/'

print('load test sub file...')
df_test = pd.read_csv('/opt/datasets/kaggle/Carvana/sample_submission.csv')
ids_test = df_test['img'].map(lambda s: s.split('.')[0])

names = []
for id in ids_test:
    names.append('{}.jpg'.format(id))


# https://www.kaggle.com/stainsby/fast-tested-rle
def run_length_encode(mask):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    inds = mask.flatten()
    runs = np.where(inds[1:] != inds[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    rle = ' '.join([str(r) for r in runs])
    return rle

# model.load_weights('/home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/best_weights_1280x1280_unet_tf_usv_crop_t0053b.hdf5')

rles = []

print('load model weights...')

# model.load_weights(filepath=weight_path + 'best_weights_1280x1280_unet_tf_usv_crop_t0053b.hdf5')
graph = tf.get_default_graph()

print('model is ready...')

q_size = 10


def data_loader(q, ):
    for start in range(0, len(ids_test), batch_size):
        x_batch = []
        end = min(start + batch_size, len(ids_test))
        ids_test_batch = ids_test[start:end]
        for id in ids_test_batch.values:
            
            img_path = '/opt/datasets/kaggle/Carvana/test/{}.jpg'.format(id)
            usv_path = '/opt/datasets/kaggle/Carvana/backgdusv2/{}.png'.format(id)
            
            img = cv2.imread(img_path)
            
#             img = cv2.fastNlMeansDenoisingColored(img,None,5,5,5,5)
            img = cv2.resize(img, (input_size_w, input_size_h),cv2.INTER_AREA)
            img = cv2.copyMakeBorder(img,64,64,64,64,cv2.BORDER_REFLECT_101)
        
            usv = cv2.imread(usv_path, cv2.IMREAD_GRAYSCALE)
            usv = cv2.resize(usv, (input_size_w, input_size_h),cv2.INTER_AREA)
            usv = cv2.copyMakeBorder(usv,64,64,64,64,cv2.BORDER_REFLECT_101)
            
            img = np.dstack((img, usv))
            
            x_batch.append(img)
        x_batch = np.array(x_batch, np.float32) /255. - .5
        q.put(x_batch)


def predictor(q, ):
    for i in tqdm(range(0, len(ids_test), batch_size)):
        x_batch = q.get()
        with graph.as_default():
            preds1 = model1.predict(x_batch, batch_size=batch_size)
            preds2 = model2.predict(x_batch, batch_size=batch_size)
            preds3 = model3.predict(x_batch, batch_size=batch_size)
            preds4 = model4.predict(x_batch, batch_size=batch_size)
            preds5 = model5.predict(x_batch, batch_size=batch_size)
            preds = (preds1 + preds2 + preds3 + preds4 + preds5)/5.
        preds = np.squeeze(preds, axis=3)
        for pred in preds:
            prob = cv2.resize(pred, (orig_width, orig_height), cv2.INTER_LINEAR)
            mask = prob > .5 ##################
            rle = run_length_encode(mask)
            rles.append(rle)


q = queue.Queue(maxsize=q_size)
t1 = threading.Thread(target=data_loader, name='DataLoader', args=(q,))
t2 = threading.Thread(target=predictor, name='Predictor', args=(q,))
print('Predicting on {} samples with batch_size = {} '.format(len(ids_test), batch_size))
t1.start()
t2.start()
# Wait for both threads to finish
t1.join()
t2.join()

print("Generating submission file...")
df = pd.DataFrame({'img': names, 'rle_mask': rles})
df.to_csv('./submit/submission0053b_1024tfusv_9970_5fold.csv.gz', index=False, compression='gzip')
print("C'est FINI !!!...")

  0%|          | 0/33355 [00:00<?, ?it/s]

load test sub file...
load model weights...
model is ready...
Predicting on 100064 samples with batch_size = 3 


  4%|▍         | 1370/33355 [45:54<17:52:08,  2.01s/it]

In [14]:
### cross validation ### unfinished

In [21]:
import os
import datetime

In [38]:
MODELS_PATH = '/home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/'
HISTORY_FOLDER_PATH = '/home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/historycv/'
nb_folds = 5
batch_size = 3

def train_single_model(num_fold, trn_index, val_index, files):
    from keras.callbacks import EarlyStopping, ModelCheckpoint, Callback
    from keras.optimizers import Adam, SGD

    print('Creating and compiling UNET...')
    
    model.load_weights('/home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/best_weights_1280x1280_unet_tf_usv_crop_t0053b.hdf5')


    final_model_path = MODELS_PATH + 'unettfcv1280x1289_crop_fold_{}.hdf5'.format(num_fold)
    if os.path.isfile(final_model_path):
        print('Model already exists for fold {}. Skipping !!!!!!!!!!'.format(final_model_path))
        return 0.0

    cache_model_path = MODELS_PATH + 'unettfcv1280x1289_crop_fold_{}.hdf5'.format(num_fold)
    if os.path.isfile(cache_model_path) and restore:
        print('Load model from last point: ', cache_model_path)
        model.load_weights(cache_model_path)
    else:
        print('Start training from begining')

#     if optim_type == 'SGD':
#         optim = SGD(lr=learning_rate, decay=1e-6, momentum=0.9, nesterov=True)
#     else:
#         optim = Adam(lr=learning_rate, decay=1e-6)
#     model.compile(optimizer=optim, loss=dice_coef_loss, metrics=[dice_coef])

    print('Fitting model...')
    df_trn = df_train.ix[trn_index, :].reset_index(drop=True)
    df_val = df_train.ix[val_index, :].reset_index(drop=True)


    print('Batch size: {}'.format(batch_size))
    samples_train_per_epoch = np.ceil(float(len(df_trn)) / float(batch_size))
    samples_valid_per_epoch = np.ceil(float(len(df_val)) / float(batch_size))
    
    print('Samples train: {}, Samples valid: {}'.format(samples_train_per_epoch, samples_valid_per_epoch))

    callbacks = [EarlyStopping(monitor='val_dice_metric',
                           patience=3,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_dice_metric',
                               factor=0.1,
                               patience=2,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
        ModelCheckpoint(cache_model_path, monitor='val_dice_metric', save_best_only=True, verbose=0)]

    
    training_gen = train_generator(df_trn)
    validation_gen = valid_generator(df_val)

    from keras.optimizers import RMSprop, Adam
    model.optimizer=Adamaccum(lr=1e-6, decay=1e-7, accumulator=10)

    history = model.fit_generator(generator=train_generator(df_trn),
                    steps_per_epoch=np.ceil(float(len(df_trn)) / float(batch_size)),
                    epochs=6,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(df_val),
                    validation_steps=np.ceil(float(len(df_val)) / float(batch_size)))

    min_loss = min(history.history['val_loss'])
    print('Minimum loss for given fold: ', min_loss)
    model.load_weights(cache_model_path)
    model.save(final_model_path)
    now = datetime.datetime.now()
#     filename = HISTORY_FOLDER_PATH + 'history_{}_{:.4f}_lr_{}_{}.csv'.format(num_fold, min_loss, learning_rate, now.strftime("%Y-%m-%d-%H-%M"))
#     pd.DataFrame(history.history).to_csv(filename, index=False)
    return min_loss


def run_cross_validation_create_models_unet(nfolds=5):
    from sklearn.model_selection import KFold

    files = df_train

    kf = KFold(n_splits=nfolds, shuffle=False, random_state=66)
    num_fold = 0
    sum_score = 0
    for trn_index, val_index in kf.split(range(len(files))):
        num_fold += 1
        print('Start KFold number {} from {}'.format(num_fold, nfolds))
        print('Split train: ', len(trn_index))
        print('Split valid: ', len(val_index))
        score = train_single_model(num_fold, trn_index, val_index, files)
        print score
        sum_score += score

    print('Avg loss: {}'.format(sum_score/nfolds))
    
    return sum_score/nfolds

In [39]:
run_cross_validation_create_models_unet()

Start KFold number 1 from 5
('Split train: ', 4070)
('Split valid: ', 1018)
Creating and compiling UNET...
Model already exists for fold /home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/unettfcv1280x1289_crop_fold_1.hdf5. Skipping !!!!!!!!!!
0.0
Start KFold number 2 from 5
('Split train: ', 4070)
('Split valid: ', 1018)
Creating and compiling UNET...
Model already exists for fold /home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/unettfcv1280x1289_crop_fold_2.hdf5. Skipping !!!!!!!!!!
0.0
Start KFold number 3 from 5
('Split train: ', 4070)
('Split valid: ', 1018)
Creating and compiling UNET...
Model already exists for fold /home/xavierc/Documents/Carvana/Kaggle-Carvana-Image-Masking-Challenge-master/weights/unettfcv1280x1289_crop_fold_3.hdf5. Skipping !!!!!!!!!!
0.0
Start KFold number 4 from 5
('Split train: ', 4071)
('Split valid: ', 1017)
Creating and compiling UNET...
Model already exists for fold /home/xavierc/

KeyboardInterrupt: 